In [2]:
import pandas as pd
import FWI
import matplotlib.pyplot as plt
import numpy as np

def total_precipitation(initial_date, final_date, dmet):
    dmet['hourly.time'] = pd.to_datetime(dmet['hourly.time'])
    dmet_copy = dmet.copy()
    dmet_copy.set_index('hourly.time', inplace=True)
    selected_rows = dmet_copy.loc[initial_date:final_date]
    total_precipitation = selected_rows['hourly.precipitation'].sum()

    return total_precipitation

In [15]:
dmet = pd.read_csv('Samples/2015/2015.csv')
start_index = 0 + 12
LAT = 40.20875
FFMCPrev = 94.29084014892578
DMCPrev = 65.2578125
DCPrev = 370.92425537109375

mts = {5: 31,
       6: 30,
       7: 31,
       8:31, 
       9:30, 
       10:31}

fwix_values = [None] * len(dmet)
ffmc_values = [None] * len(dmet)
dmc_values = [None] * len(dmet)
dc_values = [None] * len(dmet)
isi_values = [None] * len(dmet)
bui_values = [None] * len(dmet)

for i in range(start_index, len(dmet)):
    iso_timestamp = dmet.iloc[i]['hourly.time']
    print(iso_timestamp)
    MONTH = iso_timestamp.split('-')[1]
    DAY = iso_timestamp.split('-')[2].split('T')[0]
    #HOUR = iso_timestamp.split('T')[1].split(':')[0]

    print(MONTH, DAY)

    TEMP = dmet.iloc[i]['hourly.temperature_2m']
    RH = dmet.iloc[i]['hourly.relative_humidity_2m']
    WIND = dmet.iloc[i]['hourly.wind_speed_10m']

    if(int(DAY) == 1):
        LAST_DAY = mts[int(MONTH) - 1]
        LAST_MONTH = str(int(MONTH) - 1)
        
        if(int(LAST_MONTH) < 10):
            LAST_MONTH = '0' + str(int(LAST_MONTH))
            
        initial_date = f'2015-{LAST_MONTH}-{LAST_DAY}T12:00'
        final_date = f'2015-{MONTH}-{DAY}T12:00'

    elif((int(DAY) < 10) and (int(DAY) > 1)):
        id = '0' + str(int(DAY)-1)
        fd = '0' + str(int(DAY))

        initial_date = f'2015-{MONTH}-{id}T12:00'
        final_date = f'2015-{MONTH}-{fd}T12:00'
    else:
        id = str(int(DAY)-1)
        fd = str(int(DAY))

        initial_date = f'2015-{MONTH}-{id}T12:00'
        final_date = f'2015-{MONTH}-{fd}T12:00'

    
    RAIN = total_precipitation(initial_date, final_date, dmet)
    
    FFMCPrev = FWI.FFMC(TEMP,RH,WIND,RAIN,FFMCPrev)
    DMCPrev = FWI.DMC(TEMP,RH,RAIN,DMCPrev,LAT,int(MONTH))
    DCPrev = FWI.DC(TEMP,RAIN,DCPrev,LAT,int(MONTH))
    isi = FWI.ISI(WIND, FFMCPrev)
    bui = FWI.BUI(DMCPrev,DCPrev)
    fwix = FWI.FWI(isi, bui)

    fwix_values[i] = fwix
    ffmc_values[i] = FFMCPrev
    dmc_values[i] = DMCPrev
    dc_values[i] = DCPrev
    isi_values[i] = isi
    bui_values[i] = bui


dmet['fwix'] = fwix_values
dmet['ffmc'] = ffmc_values
dmet['dmc'] = dmc_values
dmet['dc'] = dc_values
dmet['isi'] = isi_values
dmet['bui'] = bui_values


dmet.to_csv('Samples/2015/2015MidDayFWIX.csv', index=False)

2015-06-20T12:00
06 20
2015-06-20 13:00:00


AttributeError: 'Timestamp' object has no attribute 'split'